In [2]:
!pip install langchain


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
import faiss
import os

google_api_key = os.getenv("GOOGLE_GENAI_API_KEY")
# Assuming you have set up authentication, no need for API key here.
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=google_api_key)


# Function to chunk and embed the text:
def chunk_and_embed_text(text, chunk_size=1000, chunk_overlap=200):
    """
    Chunk the text into smaller pieces using LangChain and generate embeddings for each chunk.
    """
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    text_chunks = text_splitter.split_text(text)

    print(f"Text has been split into {len(text_chunks)} chunks.")

    all_embeddings = []

    # Loop through each chunk and generate embeddings
    for i, chunk in enumerate(text_chunks):
        print(f"Generating embedding for chunk {i+1}/{len(text_chunks)} ...")
        embedding = embeddings.embed_query(chunk)
        all_embeddings.extend(embedding) # Accumulate all individual embeddings

    return all_embeddings

# Function to create and save the FAISS index:
def create_faiss_index(embeddings, index_file_path):
    """
    Create a FAISS index and store the embeddings.
    Save the index to a file.
    """
    embedding_dim = len(embeddings[0])  # Get the dimension of the embedding
    index = faiss.IndexFlatL2(embedding_dim)  # L2 distance metric

    # Convert the embeddings to a NumPy array and add to the index
    embeddings_np = np.array(embeddings).astype('float32')
    index.add(embeddings_np)

    # Save the FAISS index to a file
    faiss.write_index(index, index_file_path)
    print(f"FAISS index saved to: {index_file_path}")


# Example Usage:
import numpy as np
#1.  Read text from your file (replace 'your_file.txt' with your file path):
with open('requirements.txt', 'r') as file:
    text = file.read()

#2. Generate embeddings for the text:
text_embeddings = chunk_and_embed_text(text)

#3. Create and save the FAISS index (replace 'your_index.index' with your desired file path):
index_file_path = "rightnow.index"
create_faiss_index(text_embeddings, index_file_path)

Text has been split into 1 chunks.
Generating embedding for chunk 1/1 ...


TypeError: object of type 'float' has no len()